In [1]:
import os

In [2]:
%pwd

'c:\\Users\\HP\\Desktop\\Chicken_disease\\research'

In [3]:
os.chdir("C:\\Users\\HP\\Desktop\\Chicken_disease")
%pwd


'C:\\Users\\HP\\Desktop\\Chicken_disease'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



In [6]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories


In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config


        
        


In [14]:
import os
import urllib.request as request
import zipfile
from CNN_Classifier.utils.common import get_size
from CNN_Classifier import logger

In [16]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


#extract zip file

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

        


In [17]:
#pipeline for data ingestion

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file() 
except Exception as e:
    raise e




[2025-03-07 20:26:35,481: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-07 20:26:35,485: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-07 20:26:35,486: INFO: common: created directory at: artifacts]
[2025-03-07 20:26:35,489: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-07 20:26:48,718: INFO: 1079744514: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6933:134347:6A2D2:E1E42:67CB0925
Accept-Ranges: bytes
Date: Fri, 07 Mar 2025 14:56:39 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4726-BOM
X-Cache: MIS